In [ ]:
# ========================================================
# 0️⃣ IMPORT LIBRARIES
# ========================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# For imbalanced data
from imblearn.over_sampling import SMOTE

# SHAP for explainability
import shap


In [ ]:
# ========================================================
# 1️⃣ LOAD DATA
# ========================================================
df = pd.read_csv(r"F:\AI\Project\enhanced_fraud_dataset.csv")
df.head()
df.info()
df.isnull().sum()


In [ ]:
# ========================================================
# 2️⃣ FEATURE ENGINEERING
# ========================================================
model_df = df.copy()

model_df['has_anomaly'] = (model_df['anomalyFlags'] != 'NORMAL').astype(int)
model_df['anomaly_count'] = model_df['anomalyFlags'].apply(lambda x: len(x.split('|')) if x != 'NORMAL' else 0)

model_df['has_geo_anomaly'] = model_df['anomalyFlags'].str.contains('GEO').fillna(0).astype(int)
model_df['has_amount_anomaly'] = model_df['anomalyFlags'].str.contains('AMOUNT').fillna(0).astype(int)
model_df['has_velocity_anomaly'] = model_df['anomalyFlags'].str.contains('FREQ|SUCCESSION').fillna(0).astype(int)
model_df['has_balance_anomaly'] = model_df['anomalyFlags'].str.contains('BALANCE').fillna(0).astype(int)

categorical_columns = ['type', 'amount_category', 'risk_category']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    model_df[f'{col}_encoded'] = le.fit_transform(model_df[col].astype(str))
    label_encoders[col] = le

model_df['hour_sin'] = np.sin(2 * np.pi * model_df['hour_of_day']/24)
model_df['hour_cos'] = np.cos(2 * np.pi * model_df['hour_of_day']/24)
model_df['is_unusual_hours'] = ((model_df['hour_of_day'] < 6) | (model_df['hour_of_day'] > 22)).astype(int)


In [ ]:
# ========================================================
# 3️⃣ SELECT FEATURES & TARGET
# ========================================================
features = [
    'amount', 'amount_to_balance_ratio', 'is_round_amount',
    'hour_sin', 'hour_cos', 'is_weekend', 'is_unusual_hours',
    'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest',
    'distance_km', 'risk_score', 'anomaly_count', 'has_anomaly',
    'has_geo_anomaly', 'has_amount_anomaly', 'has_velocity_anomaly', 'has_balance_anomaly',
    'type_encoded', 'amount_category_encoded', 'risk_category_encoded'
]

features = [f for f in features if f in model_df.columns]
X = model_df[features].fillna(0)
y = model_df['isFraud']

print("Features shape:", X.shape)
print("Target distribution:", y.value_counts().to_dict())


In [ ]:
# ========================================================
# 4️⃣ TRAIN-TEST SPLIT
# ========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Training set:", X_train.shape, "Test set:", X_test.shape)
print("Fraud ratio in train:", y_train.mean()*100, "%")
print("Fraud ratio in test:", y_test.mean()*100, "%")


In [ ]:
# ========================================================
# 6️⃣ SCALE FEATURES
# ========================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# ========================================================
# 7️⃣ MODEL TRAINING
# ========================================================
model = RandomForestClassifier(
    n_estimators=150,
    max_depth=12,
    min_samples_split=15,
    min_samples_leaf=8,
    max_features='sqrt',
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)

model.fit(X_train_scaled, y_train_res)


In [ ]:
# ========================================================
# 8️⃣ MODEL EVALUATION
# ========================================================
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:,1]

accuracy = (y_pred == y_test).mean()
precision = (y_pred & y_test).sum() / (y_pred.sum() + 1e-8)
recall = (y_pred & y_test).sum() / (y_test.sum() + 1e-8)
f1 = 2 * (precision*recall) / (precision+recall+1e-8)
auc_score = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC Score: {auc_score:.4f}")

print(classification_report(y_test, y_pred, target_names=['Legitimate','Fraud']))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted Legit','Predicted Fraud'],
            yticklabels=['Actual Legit','Actual Fraud'])
plt.show()


In [ ]:
# ========================================================
# 9️⃣ FEATURE IMPORTANCE
# ========================================================
feature_imp = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12,8))
sns.barplot(data=feature_imp.head(15), x='importance', y='feature', palette='viridis')
plt.title('Top 15 Feature Importances')
plt.show()


In [ ]:
# ========================================================
# 🔟 SAVE MODEL & ARTIFACTS
# ========================================================
model_artifacts = {
    'model': model,
    'features': features,
    'scaler': scaler,
    'feature_importance': feature_imp,
}

joblib.dump(model_artifacts, 'anomaly_detection_model.pkl')
print("Model saved: 'anomaly_detection_model.pkl'")


In [ ]:
# ========================================================
# 1️⃣1️⃣ PREDICTION FUNCTION
# ========================================================
def predict_anomaly_transaction(new_data, model_path='anomaly_detection_model.pkl', threshold=0.5):
    artifacts = joblib.load(model_path)
    model = artifacts['model']
    features = artifacts['features']
    scaler = artifacts['scaler']
    
    X_new = new_data[features].fillna(0)
    X_new_scaled = scaler.transform(X_new)
    
    probabilities = model.predict_proba(X_new_scaled)[:,1]
    predictions = (probabilities >= threshold).astype(int)
    
    results = pd.DataFrame({
        'anomaly_probability': probabilities,
        'is_anomaly_predicted': predictions,
        'risk_level': pd.cut(probabilities, bins=[0,0.2,0.5,0.8,1.0],
                             labels=['Low','Medium','High','Critical'])
    })
    
    return results


In [ ]:
# ========================================================
# 1️⃣2️⃣ TEST PREDICTION FUNCTION
# ========================================================
sample_data = X_test.iloc[:10].copy()
predictions = predict_anomaly_transaction(sample_data)
print(predictions)


In [ ]:
# ========================================================
# 1️⃣3️⃣ SHAP EXPLAINABILITY
# ========================================================
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test_scaled)

shap.summary_plot(shap_values[1], X_test, feature_names=features, plot_type="bar")
shap.summary_plot(shap_values[1], X_test, feature_names=features)

sample_idx = 0
shap.initjs()
shap.force_plot(
    explainer.expected_value[1],
    shap_values[1][sample_idx,:],
    X_test.iloc[sample_idx,:],
    feature_names=features
)


In [ ]:
# ========================================================
# 1️⃣4️⃣ AUTOMATIC HIGH-RISK TRANSACTION DETECTION
# ========================================================
def detect_high_risk_transactions(file_path, model_path='anomaly_detection_model.pkl', threshold=0.5):
    new_data = pd.read_csv(file_path)
    print(f"Loaded {len(new_data)} transactions from {file_path}")
    
    artifacts = joblib.load(model_path)
    model = artifacts['model']
    features = artifacts['features']
    scaler = artifacts['scaler']
    
    for col in features:
        if col not in new_data.columns:
            new_data[col] = 0
    
    X_new = new_data[features].fillna(0)
    X_new_scaled = scaler.transform(X_new)
    
    probabilities = model.predict_proba(X_new_scaled)[:,1]
    predictions = (probabilities >= threshold).astype(int)
    
    risk_levels = pd.cut(probabilities, bins=[0,0.2,0.5,0.8,1.0],
                         labels=['Low','Medium','High','Critical'])
    
    results = new_data.copy()
    results['anomaly_probability'] = probabilities
    results['is_anomaly_predicted'] = predictions
    results['risk_level'] = risk_levels
    
    high_risk = results[results['risk_level'].isin(['High','Critical'])].sort_values('anomaly_probability', ascending=False)
    
    print(f"Detected {len(high_risk)} high/critical risk transactions")
    return high_risk


TEST HIGH-RISK DETECTION

In [ ]:
file_path = r"F:\AI\Project\new_transactions.csv"
high_risk_transactions = detect_high_risk_transactions(file_path)
high_risk_transactions.head(10)


In [ ]:
EXPORT HIGH-RISK TRANSACTIONS TO CSV

In [ ]:
import datetime

def export_high_risk_transactions(high_risk_df, output_dir=r"F:\AI\Project\Reports"):
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"high_risk_transactions_{timestamp}.csv"
    file_path = os.path.join(output_dir, file_name)
    
    high_risk_df.to_csv(file_path, index=False)
    print(f"High-risk transactions exported to: {file_path}")
    return file_path


TEST EXPORT FUNCTION

In [ ]:
exported_file = export_high_risk_transactions(high_risk_transactions)